# FIA-IBMS Oracle Database Setup
**Target:** `ibms_user@localhost:1521/FREEPDB1`  
**Tables:** 20 | **Rows:** ~1.56M  
Run each cell in order.

## Cell 1: Imports & Configuration

In [6]:
import oracledb
import pandas as pd
import numpy as np
import time
from pathlib import Path

ORACLE_USER = "ibms_user"
ORACLE_PASS = "ibms_pass"
ORACLE_DSN  = "localhost:1521/FREEPDB1"

DATA_DIR = Path.home() / "ml-projects" / "python-projects" / "IBMS_LLM" / "data" / "raw"

print(f"Data dir: {DATA_DIR}")
print(f"CSV files: {len(list(DATA_DIR.glob('*.csv')))}")

Data dir: /home/maliciit/ml-projects/python-projects/IBMS_LLM/data/raw
CSV files: 20


## Cell 2: Test Oracle Connection

In [7]:
conn = oracledb.connect(user=ORACLE_USER, password=ORACLE_PASS, dsn=ORACLE_DSN)
cursor = conn.cursor()
cursor.execute("SELECT 1 FROM dual")
print("Connection successful:", cursor.fetchone())
cursor.execute("SELECT SYS_CONTEXT('USERENV','CURRENT_SCHEMA') FROM dual")
print("Schema:", cursor.fetchone()[0])
cursor.execute("SELECT banner FROM v$version WHERE ROWNUM = 1")
print("Oracle version:", cursor.fetchone()[0])

Connection successful: (1,)
Schema: IBMS_USER
Oracle version: Oracle AI Database 26ai Free Release 23.26.1.0.0 - Develop, Learn, and Run for Free


## Cell 3: Drop All Existing Tables (safe — ignores if not exist)

In [8]:
DROP_ORDER = [
    "audit_log", "suspect_networks", "risk_profiles", "offloading_records",
    "illegal_crossings", "trafficking_cases", "ecl_entries", "watchlist",
    "family_relationships", "detention_records", "removal_orders",
    "asylum_claims", "travel_records", "visa_applications",
    "document_registry", "sponsors", "travelers", "visa_categories",
    "ports_of_entry", "countries"
]

cursor = conn.cursor()
for table in DROP_ORDER:
    try:
        cursor.execute(f"DROP TABLE {table} CASCADE CONSTRAINTS")
        print(f"  Dropped: {table}")
    except oracledb.DatabaseError:
        print(f"  Skip (not exists): {table}")

conn.commit()
print("\nDone.")

  Skip (not exists): audit_log
  Skip (not exists): suspect_networks
  Skip (not exists): risk_profiles
  Skip (not exists): offloading_records
  Skip (not exists): illegal_crossings
  Skip (not exists): trafficking_cases
  Skip (not exists): ecl_entries
  Skip (not exists): watchlist
  Skip (not exists): family_relationships
  Skip (not exists): detention_records
  Skip (not exists): removal_orders
  Skip (not exists): asylum_claims
  Skip (not exists): travel_records
  Skip (not exists): visa_applications
  Skip (not exists): document_registry
  Skip (not exists): sponsors
  Skip (not exists): travelers
  Skip (not exists): visa_categories
  Dropped: ports_of_entry
  Dropped: countries

Done.


## Cell 4: Create Table — countries

In [9]:
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE countries (
    country_id          NUMBER(10) PRIMARY KEY,
    country_name        VARCHAR2(100) NOT NULL,
    iso3_code           VARCHAR2(3) NOT NULL,
    region              VARCHAR2(50) NOT NULL,
    sub_region          VARCHAR2(50),
    income_group        VARCHAR2(30),
    is_conflict_zone    NUMBER(1) DEFAULT 0,
    CONSTRAINT uq_countries_iso3 UNIQUE (iso3_code)
)
""")
conn.commit()
print("Created: countries")

Created: countries


## Cell 5: Create Table — ports_of_entry

In [10]:
cursor.execute("""
CREATE TABLE ports_of_entry (
    port_id             NUMBER(10) PRIMARY KEY,
    port_name           VARCHAR2(150) NOT NULL,
    port_type           VARCHAR2(20) NOT NULL,
    city                VARCHAR2(100),
    country_id          NUMBER(10) NOT NULL,
    iata_code           VARCHAR2(10),
    latitude            NUMBER(9,6),
    longitude           NUMBER(9,6),
    is_active           NUMBER(1) DEFAULT 1,
    CONSTRAINT chk_port_type CHECK (port_type IN ('Airport','Land Border','Seaport')),
    CONSTRAINT fk_port_country FOREIGN KEY (country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: ports_of_entry")

Created: ports_of_entry


## Cell 6: Create Table — visa_categories

In [11]:
cursor.execute("""
CREATE TABLE visa_categories (
    visa_code           VARCHAR2(5) PRIMARY KEY,
    visa_name           VARCHAR2(100) NOT NULL,
    visa_class          VARCHAR2(20) NOT NULL,
    max_duration_days   NUMBER(10),
    allows_work         NUMBER(1) DEFAULT 0,
    allows_study        NUMBER(1) DEFAULT 0,
    requires_sponsor    NUMBER(1) DEFAULT 0,
    description         VARCHAR2(255),
    CONSTRAINT chk_visa_class CHECK (visa_class IN ('Immigrant','Non-Immigrant','Refugee','Transit','Diplomatic'))
)
""")
conn.commit()
print("Created: visa_categories")

Created: visa_categories


## Cell 7: Create Table — travelers

In [12]:
cursor.execute("""
CREATE TABLE travelers (
    traveler_id         NUMBER(10) PRIMARY KEY,
    passport_number     VARCHAR2(20) NOT NULL,
    first_name          VARCHAR2(100) NOT NULL,
    last_name           VARCHAR2(100) NOT NULL,
    date_of_birth       DATE NOT NULL,
    gender              CHAR(1),
    nationality_id      NUMBER(10) NOT NULL,
    birth_country_id    NUMBER(10) NOT NULL,
    occupation          VARCHAR2(100),
    education_level     VARCHAR2(20),
    marital_status      VARCHAR2(10),
    phone_number        VARCHAR2(20),
    email               VARCHAR2(150),
    created_at          TIMESTAMP DEFAULT SYSTIMESTAMP,
    CONSTRAINT uq_travelers_passport UNIQUE (passport_number),
    CONSTRAINT chk_gender CHECK (gender IN ('M','F','X')),
    CONSTRAINT chk_education CHECK (education_level IN ('None','Primary','Secondary','Bachelors','Masters','Doctorate','Professional')),
    CONSTRAINT chk_marital CHECK (marital_status IN ('Single','Married','Divorced','Widowed')),
    CONSTRAINT fk_traveler_nationality FOREIGN KEY (nationality_id) REFERENCES countries(country_id),
    CONSTRAINT fk_traveler_birth_country FOREIGN KEY (birth_country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: travelers")

Created: travelers


## Cell 8: Create Table — sponsors

In [13]:
cursor.execute("""
CREATE TABLE sponsors (
    sponsor_id          NUMBER(10) PRIMARY KEY,
    sponsor_type        VARCHAR2(20) NOT NULL,
    sponsor_name        VARCHAR2(200) NOT NULL,
    country_id          NUMBER(10) NOT NULL,
    city                VARCHAR2(100),
    industry            VARCHAR2(100),
    registration_number VARCHAR2(50),
    is_verified         NUMBER(1) DEFAULT 0,
    CONSTRAINT chk_sponsor_type CHECK (sponsor_type IN ('Employer','Family','Institution','Government','Self')),
    CONSTRAINT fk_sponsor_country FOREIGN KEY (country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: sponsors")

Created: sponsors


## Cell 9: Create Table — document_registry

In [14]:
cursor.execute("""
CREATE TABLE document_registry (
    document_id         NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    document_type       VARCHAR2(30) NOT NULL,
    document_number     VARCHAR2(50) NOT NULL,
    issuing_country_id  NUMBER(10) NOT NULL,
    issue_date          DATE NOT NULL,
    expiry_date         DATE,
    status              VARCHAR2(20) NOT NULL,
    CONSTRAINT chk_doc_type CHECK (document_type IN ('Passport','CNIC','NICOP','Travel Document','Refugee Card','Laissez-Passer')),
    CONSTRAINT chk_doc_status CHECK (status IN ('Valid','Expired','Cancelled','Reported Stolen','Suspended','Under Review')),
    CONSTRAINT fk_doc_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_doc_country FOREIGN KEY (issuing_country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: document_registry")

Created: document_registry


## Cell 10: Create Table — visa_applications

In [15]:
cursor.execute("""
CREATE TABLE visa_applications (
    application_id      NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    visa_code           VARCHAR2(5) NOT NULL,
    sponsor_id          NUMBER(10),
    application_date    DATE NOT NULL,
    decision_date       DATE,
    status              VARCHAR2(15) NOT NULL,
    denial_reason       VARCHAR2(255),
    processing_office   VARCHAR2(100),
    fee_amount          NUMBER(10,2),
    fee_currency        VARCHAR2(3) DEFAULT 'PKR',
    CONSTRAINT chk_visa_status CHECK (status IN ('Pending','Approved','Denied','Revoked','Expired','Withdrawn')),
    CONSTRAINT fk_visa_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_visa_code FOREIGN KEY (visa_code) REFERENCES visa_categories(visa_code),
    CONSTRAINT fk_visa_sponsor FOREIGN KEY (sponsor_id) REFERENCES sponsors(sponsor_id)
)
""")
conn.commit()
print("Created: visa_applications")

Created: visa_applications


## Cell 11: Create Table — travel_records

In [16]:
cursor.execute("""
CREATE TABLE travel_records (
    record_id           NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    application_id      NUMBER(10),
    entry_port_id       NUMBER(10),
    exit_port_id        NUMBER(10),
    entry_date          TIMESTAMP,
    exit_date           TIMESTAMP,
    travel_direction    VARCHAR2(10) NOT NULL,
    travel_purpose      VARCHAR2(30),
    flight_number       VARCHAR2(15),
    carrier             VARCHAR2(100),
    declared_stay_days  NUMBER(10),
    actual_stay_days    NUMBER(10),
    overstay_flag       NUMBER(1) DEFAULT 0,
    flagged_suspicious  NUMBER(1) DEFAULT 0,
    referral_to_investigation NUMBER(1) DEFAULT 0,
    CONSTRAINT chk_travel_dir CHECK (travel_direction IN ('Inbound','Outbound','Transit')),
    CONSTRAINT fk_travel_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_travel_app FOREIGN KEY (application_id) REFERENCES visa_applications(application_id),
    CONSTRAINT fk_travel_entry_port FOREIGN KEY (entry_port_id) REFERENCES ports_of_entry(port_id),
    CONSTRAINT fk_travel_exit_port FOREIGN KEY (exit_port_id) REFERENCES ports_of_entry(port_id)
)
""")
conn.commit()
print("Created: travel_records")

Created: travel_records


## Cell 12: Create Table — asylum_claims

In [17]:
cursor.execute("""
CREATE TABLE asylum_claims (
    claim_id            NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    filing_date         DATE NOT NULL,
    claim_basis         VARCHAR2(20) NOT NULL,
    origin_country_id   NUMBER(10) NOT NULL,
    status              VARCHAR2(25) NOT NULL,
    decision_date       DATE,
    assigned_officer    VARCHAR2(100),
    dependents_count    NUMBER(10) DEFAULT 0,
    CONSTRAINT chk_asylum_basis CHECK (claim_basis IN ('Persecution','War','Political','Religious','Ethnic','Gender-Based','Other')),
    CONSTRAINT chk_asylum_status CHECK (status IN ('Filed','Under Review','Interview Scheduled','Approved','Denied','Appealed','Withdrawn')),
    CONSTRAINT fk_asylum_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_asylum_country FOREIGN KEY (origin_country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: asylum_claims")

Created: asylum_claims


## Cell 13: Create Table — removal_orders

In [18]:
cursor.execute("""
CREATE TABLE removal_orders (
    order_id            NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    order_date          DATE NOT NULL,
    reason              VARCHAR2(30) NOT NULL,
    destination_country_id NUMBER(10) NOT NULL,
    execution_date      DATE,
    status              VARCHAR2(15) NOT NULL,
    voluntary_departure NUMBER(1) DEFAULT 0,
    CONSTRAINT chk_removal_reason CHECK (reason IN ('Visa Overstay','Criminal Offense','Security Threat','Fraudulent Documents','Unauthorized Work','Failed Asylum','Trafficking Involvement','Other')),
    CONSTRAINT chk_removal_status CHECK (status IN ('Issued','Appealed','Executed','Stayed','Cancelled')),
    CONSTRAINT fk_removal_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_removal_country FOREIGN KEY (destination_country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: removal_orders")

Created: removal_orders


## Cell 14: Create Table — detention_records

In [19]:
cursor.execute("""
CREATE TABLE detention_records (
    detention_id        NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    facility_name       VARCHAR2(150) NOT NULL,
    facility_country_id NUMBER(10) NOT NULL,
    intake_date         DATE NOT NULL,
    release_date        DATE,
    reason              VARCHAR2(255),
    removal_order_id    NUMBER(10),
    status              VARCHAR2(15),
    CONSTRAINT chk_detention_status CHECK (status IN ('Active','Released','Transferred','Deported')),
    CONSTRAINT fk_detention_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_detention_country FOREIGN KEY (facility_country_id) REFERENCES countries(country_id),
    CONSTRAINT fk_detention_order FOREIGN KEY (removal_order_id) REFERENCES removal_orders(order_id)
)
""")
conn.commit()
print("Created: detention_records")

Created: detention_records


## Cell 15: Create Table — family_relationships

In [20]:
cursor.execute("""
CREATE TABLE family_relationships (
    relationship_id     NUMBER(10) PRIMARY KEY,
    traveler_id_1       NUMBER(10) NOT NULL,
    traveler_id_2       NUMBER(10) NOT NULL,
    relationship_type   VARCHAR2(15) NOT NULL,
    verified            NUMBER(1) DEFAULT 0,
    CONSTRAINT chk_rel_type CHECK (relationship_type IN ('Spouse','Parent','Child','Sibling','Grandparent','Grandchild','Other')),
    CONSTRAINT fk_rel_traveler1 FOREIGN KEY (traveler_id_1) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_rel_traveler2 FOREIGN KEY (traveler_id_2) REFERENCES travelers(traveler_id)
)
""")
conn.commit()
print("Created: family_relationships")

Created: family_relationships


## Cell 16: Create Table — watchlist

In [21]:
cursor.execute("""
CREATE TABLE watchlist (
    alert_id            NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10),
    passport_number     VARCHAR2(20),
    alert_type          VARCHAR2(25) NOT NULL,
    severity            VARCHAR2(10),
    issued_by           VARCHAR2(100),
    issued_date         DATE NOT NULL,
    expiry_date         DATE,
    is_active           NUMBER(1) DEFAULT 1,
    notes               VARCHAR2(500),
    CONSTRAINT chk_alert_type CHECK (alert_type IN ('Security','Criminal','Immigration Violation','Interpol','Lost Document','Fraud','Trafficking','Smuggling','ECL')),
    CONSTRAINT chk_severity CHECK (severity IN ('Low','Medium','High','Critical')),
    CONSTRAINT fk_watchlist_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id)
)
""")
conn.commit()
print("Created: watchlist")

Created: watchlist


## Cell 17: Create Table — ecl_entries

In [22]:
cursor.execute("""
CREATE TABLE ecl_entries (
    ecl_id              NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    reason              VARCHAR2(30) NOT NULL,
    issuing_authority   VARCHAR2(100) NOT NULL,
    issued_date         DATE NOT NULL,
    expiry_date         DATE,
    status              VARCHAR2(10) NOT NULL,
    case_reference      VARCHAR2(50),
    CONSTRAINT chk_ecl_reason CHECK (reason IN ('Financial Crime','Court Order','National Security','Terrorism','Tax Evasion','Money Laundering','Pending Investigation','Other')),
    CONSTRAINT chk_ecl_status CHECK (status IN ('Active','Lifted','Expired','Under Review')),
    CONSTRAINT fk_ecl_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id)
)
""")
conn.commit()
print("Created: ecl_entries")

Created: ecl_entries


## Cell 18: Create Table — trafficking_cases

In [23]:
cursor.execute("""
CREATE TABLE trafficking_cases (
    case_id             NUMBER(10) PRIMARY KEY,
    case_type           VARCHAR2(20) NOT NULL,
    victim_traveler_id  NUMBER(10),
    suspect_traveler_id NUMBER(10),
    origin_country_id   NUMBER(10) NOT NULL,
    transit_country_id  NUMBER(10),
    destination_country_id NUMBER(10) NOT NULL,
    reported_date       DATE NOT NULL,
    status              VARCHAR2(25) NOT NULL,
    victim_age_at_time  NUMBER(10),
    victim_gender       CHAR(1),
    CONSTRAINT chk_trafficking_type CHECK (case_type IN ('Labor','Sexual','Organ','Forced Marriage','Child','Domestic Servitude','Debt Bondage','Other')),
    CONSTRAINT chk_trafficking_status CHECK (status IN ('Open','Under Investigation','Prosecuted','Convicted','Acquitted','Closed')),
    CONSTRAINT fk_trafficking_victim FOREIGN KEY (victim_traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_trafficking_suspect FOREIGN KEY (suspect_traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_trafficking_origin FOREIGN KEY (origin_country_id) REFERENCES countries(country_id),
    CONSTRAINT fk_trafficking_transit FOREIGN KEY (transit_country_id) REFERENCES countries(country_id),
    CONSTRAINT fk_trafficking_dest FOREIGN KEY (destination_country_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: trafficking_cases")

Created: trafficking_cases


## Cell 19: Create Table — illegal_crossings

In [24]:
cursor.execute("""
CREATE TABLE illegal_crossings (
    crossing_id         NUMBER(10) PRIMARY KEY,
    detected_date       TIMESTAMP NOT NULL,
    location_name       VARCHAR2(150),
    nearest_port_id     NUMBER(10),
    latitude            NUMBER(9,6),
    longitude           NUMBER(9,6),
    direction           VARCHAR2(10),
    detection_method    VARCHAR2(20) NOT NULL,
    group_size          NUMBER(10) DEFAULT 1,
    smuggler_linked     NUMBER(1) DEFAULT 0,
    apprehended_count   NUMBER(10) DEFAULT 0,
    outcome             VARCHAR2(20) NOT NULL,
    lead_traveler_id    NUMBER(10),
    nationality_id      NUMBER(10),
    CONSTRAINT chk_crossing_dir CHECK (direction IN ('Inbound','Outbound')),
    CONSTRAINT chk_detection CHECK (detection_method IN ('Patrol','Sensor','Tip-Off','Drone','CCTV','Informant','Checkpoint','Other')),
    CONSTRAINT chk_outcome CHECK (outcome IN ('Apprehended','Escaped','Turned Back','Partial Capture')),
    CONSTRAINT fk_crossing_port FOREIGN KEY (nearest_port_id) REFERENCES ports_of_entry(port_id),
    CONSTRAINT fk_crossing_traveler FOREIGN KEY (lead_traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_crossing_nationality FOREIGN KEY (nationality_id) REFERENCES countries(country_id)
)
""")
conn.commit()
print("Created: illegal_crossings")

Created: illegal_crossings


## Cell 20: Create Table — offloading_records

In [25]:
cursor.execute("""
CREATE TABLE offloading_records (
    offload_id          NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    offload_date        DATE NOT NULL,
    port_id             NUMBER(10) NOT NULL,
    reason              VARCHAR2(30) NOT NULL,
    airline             VARCHAR2(100),
    flight_number       VARCHAR2(15),
    destination_country_id NUMBER(10),
    linked_alert_id     NUMBER(10),
    linked_ecl_id       NUMBER(10),
    CONSTRAINT chk_offload_reason CHECK (reason IN ('Invalid Documents','Watchlist Hit','ECL Match','Court Order','Suspicious Behavior','Incomplete Travel Docs','Interpol Alert','Airline Refusal','Other')),
    CONSTRAINT fk_offload_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id),
    CONSTRAINT fk_offload_port FOREIGN KEY (port_id) REFERENCES ports_of_entry(port_id),
    CONSTRAINT fk_offload_country FOREIGN KEY (destination_country_id) REFERENCES countries(country_id),
    CONSTRAINT fk_offload_alert FOREIGN KEY (linked_alert_id) REFERENCES watchlist(alert_id),
    CONSTRAINT fk_offload_ecl FOREIGN KEY (linked_ecl_id) REFERENCES ecl_entries(ecl_id)
)
""")
conn.commit()
print("Created: offloading_records")

Created: offloading_records


## Cell 21: Create Table — risk_profiles

In [26]:
cursor.execute("""
CREATE TABLE risk_profiles (
    profile_id          NUMBER(10) PRIMARY KEY,
    traveler_id         NUMBER(10) NOT NULL,
    risk_score          NUMBER(5,2) NOT NULL,
    risk_tier           VARCHAR2(10) NOT NULL,
    overstay_history    NUMBER(1) DEFAULT 0,
    watchlist_hits      NUMBER(10) DEFAULT 0,
    conflict_zone_travel NUMBER(1) DEFAULT 0,
    document_fraud_flag NUMBER(1) DEFAULT 0,
    trafficking_association NUMBER(1) DEFAULT 0,
    criminal_record     NUMBER(1) DEFAULT 0,
    network_membership  NUMBER(1) DEFAULT 0,
    last_updated        TIMESTAMP DEFAULT SYSTIMESTAMP,
    CONSTRAINT uq_risk_traveler UNIQUE (traveler_id),
    CONSTRAINT chk_risk_score CHECK (risk_score >= 0 AND risk_score <= 100),
    CONSTRAINT chk_risk_tier CHECK (risk_tier IN ('Low','Medium','High','Critical')),
    CONSTRAINT fk_risk_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id)
)
""")
conn.commit()
print("Created: risk_profiles")

Created: risk_profiles


## Cell 22: Create Table — suspect_networks

In [27]:
cursor.execute("""
CREATE TABLE suspect_networks (
    link_id             NUMBER(10) PRIMARY KEY,
    network_id          VARCHAR2(20) NOT NULL,
    traveler_id         NUMBER(10) NOT NULL,
    role                VARCHAR2(20) NOT NULL,
    confidence_level    VARCHAR2(10),
    identified_date     DATE NOT NULL,
    is_active           NUMBER(1) DEFAULT 1,
    CONSTRAINT chk_network_role CHECK (role IN ('Organizer','Recruiter','Transporter','Financier','Document Forger','Lookout','Coordinator','Unknown')),
    CONSTRAINT chk_confidence CHECK (confidence_level IN ('Low','Medium','High','Confirmed')),
    CONSTRAINT fk_network_traveler FOREIGN KEY (traveler_id) REFERENCES travelers(traveler_id)
)
""")
conn.commit()
print("Created: suspect_networks")

Created: suspect_networks


## Cell 23: Create Table — audit_log

In [28]:
cursor.execute("""
CREATE TABLE audit_log (
    log_id              NUMBER(10) PRIMARY KEY,
    table_name          VARCHAR2(50) NOT NULL,
    record_id           NUMBER(10) NOT NULL,
    action              VARCHAR2(10) NOT NULL,
    action_timestamp    TIMESTAMP NOT NULL,
    officer_id          VARCHAR2(20) NOT NULL,
    officer_name        VARCHAR2(100),
    terminal_id         VARCHAR2(30),
    port_id             NUMBER(10),
    ip_address          VARCHAR2(45),
    details             VARCHAR2(500),
    CONSTRAINT chk_audit_action CHECK (action IN ('INSERT','UPDATE','DELETE','VIEW','EXPORT')),
    CONSTRAINT fk_audit_port FOREIGN KEY (port_id) REFERENCES ports_of_entry(port_id)
)
""")
conn.commit()
print("Created: audit_log")

Created: audit_log


## Cell 24: Create All Indexes

In [29]:
indexes = [
    "CREATE INDEX idx_travelers_nationality ON travelers(nationality_id)",
    "CREATE INDEX idx_travelers_dob ON travelers(date_of_birth)",
    "CREATE INDEX idx_travel_records_traveler ON travel_records(traveler_id)",
    "CREATE INDEX idx_travel_records_entry_date ON travel_records(entry_date)",
    "CREATE INDEX idx_travel_records_overstay ON travel_records(overstay_flag)",
    "CREATE INDEX idx_visa_apps_traveler ON visa_applications(traveler_id)",
    "CREATE INDEX idx_visa_apps_status ON visa_applications(status)",
    "CREATE INDEX idx_visa_apps_date ON visa_applications(application_date)",
    "CREATE INDEX idx_asylum_status ON asylum_claims(status)",
    "CREATE INDEX idx_watchlist_active ON watchlist(is_active)",
    "CREATE INDEX idx_watchlist_type ON watchlist(alert_type)",
    "CREATE INDEX idx_ecl_status ON ecl_entries(status)",
    "CREATE INDEX idx_risk_tier ON risk_profiles(risk_tier)",
    "CREATE INDEX idx_risk_score ON risk_profiles(risk_score)",
    "CREATE INDEX idx_trafficking_status ON trafficking_cases(status)",
    "CREATE INDEX idx_illegal_crossings_date ON illegal_crossings(detected_date)",
    "CREATE INDEX idx_offloading_date ON offloading_records(offload_date)",
    "CREATE INDEX idx_document_traveler ON document_registry(traveler_id)",
    "CREATE INDEX idx_document_status ON document_registry(status)",
    "CREATE INDEX idx_audit_timestamp ON audit_log(action_timestamp)",
    "CREATE INDEX idx_audit_officer ON audit_log(officer_id)",
    "CREATE INDEX idx_suspect_network_id ON suspect_networks(network_id)",
]

for idx_sql in indexes:
    try:
        cursor.execute(idx_sql)
        idx_name = idx_sql.split("INDEX ")[1].split(" ON")[0]
        print(f"  ✓ {idx_name}")
    except oracledb.DatabaseError as e:
        if "ORA-00955" in str(e):
            print(f"  ~ Already exists: {idx_sql.split('INDEX ')[1].split(' ON')[0]}")
        else:
            print(f"  ✗ {e}")

conn.commit()
print("\nDone.")

  ✓ idx_travelers_nationality
  ✓ idx_travelers_dob
  ✓ idx_travel_records_traveler
  ✓ idx_travel_records_entry_date
  ✓ idx_travel_records_overstay
  ✓ idx_visa_apps_traveler
  ✓ idx_visa_apps_status
  ✓ idx_visa_apps_date
  ✓ idx_asylum_status
  ✓ idx_watchlist_active
  ✓ idx_watchlist_type
  ✓ idx_ecl_status
  ✓ idx_risk_tier
  ✓ idx_risk_score
  ✓ idx_trafficking_status
  ✓ idx_illegal_crossings_date
  ✓ idx_offloading_date
  ✓ idx_document_traveler
  ✓ idx_document_status
  ✓ idx_audit_timestamp
  ✓ idx_audit_officer
  ✓ idx_suspect_network_id

Done.


## Cell 25: Verify All 20 Tables Exist

In [30]:
cursor.execute("SELECT table_name FROM user_tables ORDER BY table_name")
tables = [row[0] for row in cursor.fetchall()]
print(f"Tables in schema: {len(tables)}\n")
for t in tables:
    print(f"  {t}")

assert len(tables) == 20, f"Expected 20 tables, got {len(tables)}"
print("\n✓ All 20 tables created successfully.")

Tables in schema: 20

  ASYLUM_CLAIMS
  AUDIT_LOG
  COUNTRIES
  DETENTION_RECORDS
  DOCUMENT_REGISTRY
  ECL_ENTRIES
  FAMILY_RELATIONSHIPS
  ILLEGAL_CROSSINGS
  OFFLOADING_RECORDS
  PORTS_OF_ENTRY
  REMOVAL_ORDERS
  RISK_PROFILES
  SPONSORS
  SUSPECT_NETWORKS
  TRAFFICKING_CASES
  TRAVELERS
  TRAVEL_RECORDS
  VISA_APPLICATIONS
  VISA_CATEGORIES
  WATCHLIST

✓ All 20 tables created successfully.


---
## CSV Data Loading
---

## Cell 26: Define the CSV Loader Function

In [31]:
# Columns that are DATE in Oracle
DATE_COLS = {
    "date_of_birth", "issue_date", "expiry_date", "application_date",
    "decision_date", "filing_date", "order_date", "execution_date",
    "intake_date", "release_date", "issued_date", "reported_date",
    "offload_date", "identified_date",
}

# Columns that are TIMESTAMP in Oracle
TS_COLS = {
    "created_at", "entry_date", "exit_date", "detected_date",
    "action_timestamp", "last_updated",
}

BATCH_SIZE = 5000


def load_csv(table_name, csv_filename):
    """Load a CSV file into an Oracle table with batch inserts."""
    csv_path = DATA_DIR / csv_filename
    if not csv_path.exists():
        print(f"  ⚠ SKIP: {csv_filename} not found")
        return 0

    df = pd.read_csv(csv_path, low_memory=False)
    total = len(df)
    if total == 0:
        print(f"  ⚠ SKIP: {csv_filename} empty")
        return 0

    # Parse dates/timestamps
    for col in df.columns:
        if col in DATE_COLS or col in TS_COLS:
            df[col] = pd.to_datetime(df[col], errors="coerce", format="mixed")

    # Build INSERT
    cols = df.columns.tolist()
    placeholders = ", ".join([f":{i+1}" for i in range(len(cols))])
    insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES ({placeholders})"

    cur = conn.cursor()
    loaded = 0
    errors = 0
    t0 = time.time()

    for start in range(0, total, BATCH_SIZE):
        batch_df = df.iloc[start:start + BATCH_SIZE]
        rows = []

        for _, row in batch_df.iterrows():
            row_data = []
            for col in cols:
                val = row[col]
                # Handle NaN/None
                if pd.isna(val):
                    row_data.append(None)
                elif col in DATE_COLS or col in TS_COLS:
                    row_data.append(val.to_pydatetime())
                elif isinstance(val, float) and val == int(val):
                    row_data.append(int(val))
                else:
                    row_data.append(val)
            rows.append(row_data)

        try:
            cur.executemany(insert_sql, rows)
            loaded += len(rows)
        except oracledb.DatabaseError:
            # Fallback: row-by-row
            for row_data in rows:
                try:
                    cur.execute(insert_sql, row_data)
                    loaded += 1
                except oracledb.DatabaseError as e:
                    errors += 1
                    if errors <= 3:  # Print first 3 errors
                        print(f"    Error: {str(e)[:100]}")

        pct = min(100, int((start + BATCH_SIZE) / total * 100))
        elapsed = time.time() - t0
        print(f"\r  {table_name}: {pct:3d}% | {loaded:,}/{total:,} | {elapsed:.0f}s", end="", flush=True)

    conn.commit()
    elapsed = time.time() - t0
    err_msg = f" | {errors} errors" if errors else ""
    print(f"\r  ✓ {table_name}: {loaded:,} rows | {elapsed:.1f}s{err_msg}{'':30}")
    return loaded

print("Loader function defined.")

Loader function defined.


## Cell 27: Load Reference Tables (countries, ports_of_entry, visa_categories, sponsors)

In [32]:
load_csv("countries", "countries.csv")
load_csv("ports_of_entry", "ports_of_entry.csv")
load_csv("visa_categories", "visa_categories.csv")
load_csv("sponsors", "sponsors.csv")
print("\nReference tables loaded.")

  ✓ countries: 50 rows | 0.7s                              
  ✓ ports_of_entry: 39 rows | 0.0s                              
  ✓ visa_categories: 20 rows | 0.0s                              
  ✓ sponsors: 90 rows | 0.0s                              

Reference tables loaded.


## Cell 28: Load Core Entity Tables (travelers, document_registry)

In [33]:
load_csv("travelers", "travelers.csv")
load_csv("document_registry", "document_registry.csv")
print("\nCore entity tables loaded.")

  ✓ travelers: 150,000 rows | 13.3s                              
  ✓ document_registry: 219,240 rows | 11.7s                              

Core entity tables loaded.


## Cell 29: Load Transaction Tables (visa_applications, travel_records, asylum_claims, removal_orders, detention_records)

In [34]:
load_csv("visa_applications", "visa_applications.csv")
load_csv("travel_records", "travel_records.csv")
load_csv("asylum_claims", "asylum_claims.csv")
load_csv("removal_orders", "removal_orders.csv")
load_csv("detention_records", "detention_records.csv")
print("\nTransaction tables loaded.")

  ✓ visa_applications: 250,000 rows | 20.9s                              
  ✓ travel_records: 500,000 rows | 52.4s                              
  ✓ asylum_claims: 30,000 rows | 2.1s                              
  ✓ removal_orders: 15,000 rows | 1.1s                              
  ✓ detention_records: 12,000 rows | 0.7s                              

Transaction tables loaded.


## Cell 30: Load Relationship & Flag Tables (family, watchlist, ecl)

In [35]:
load_csv("family_relationships", "family_relationships.csv")
load_csv("watchlist", "watchlist.csv")
load_csv("ecl_entries", "ecl_entries.csv")
print("\nRelationship & flag tables loaded.")

  ✓ family_relationships: 48,713 rows | 1.9s                              
  ✓ watchlist: 8,000 rows | 0.7s                              
    Error: ORA-00001: unique constraint (IBMS_USER.SYS_C008708) violated on table IBMS_USER.ECL_ENTRIES columns
    Error: ORA-00001: unique constraint (IBMS_USER.SYS_C008708) violated on table IBMS_USER.ECL_ENTRIES columns
    Error: ORA-00001: unique constraint (IBMS_USER.SYS_C008708) violated on table IBMS_USER.ECL_ENTRIES columns
  ✓ ecl_entries: 2,642 rows | 0.5s | 358 errors                              

Relationship & flag tables loaded.


## Cell 31: Load Investigation Tables (trafficking, illegal_crossings, offloading)

In [36]:
load_csv("trafficking_cases", "trafficking_cases.csv")
load_csv("illegal_crossings", "illegal_crossings.csv")
load_csv("offloading_records", "offloading_records.csv")
print("\nInvestigation tables loaded.")

  ✓ trafficking_cases: 5,000 rows | 0.3s                              
  ✓ illegal_crossings: 25,000 rows | 1.7s                              
    Error: ORA-00001: unique constraint (IBMS_USER.SYS_C008738) violated on table IBMS_USER.OFFLOADING_RECORDS 
    Error: ORA-00001: unique constraint (IBMS_USER.SYS_C008738) violated on table IBMS_USER.OFFLOADING_RECORDS 
    Error: ORA-00001: unique constraint (IBMS_USER.SYS_C008738) violated on table IBMS_USER.OFFLOADING_RECORDS 
  ✓ offloading_records: 9,492 rows | 1.4s | 508 errors                              

Investigation tables loaded.


## Cell 32: Load Analytics & System Tables (risk_profiles, suspect_networks, audit_log)

In [37]:
load_csv("risk_profiles", "risk_profiles.csv")
load_csv("suspect_networks", "suspect_networks.csv")
load_csv("audit_log", "audit_log.csv")
print("\nAnalytics & system tables loaded.")

  ✓ risk_profiles: 150,000 rows | 10.4s                              
  ✓ suspect_networks: 5,000 rows | 0.3s                              
  ✓ audit_log: 100,000 rows | 7.1s                              

Analytics & system tables loaded.


---
## Verification
---

## Cell 33: Final Verification — Row Counts for All 20 Tables

In [38]:
EXPECTED = [
    ("countries", 50),
    ("ports_of_entry", 39),
    ("visa_categories", 20),
    ("sponsors", 100),
    ("travelers", 150000),
    ("document_registry", 200000),
    ("visa_applications", 250000),
    ("travel_records", 500000),
    ("asylum_claims", 30000),
    ("removal_orders", 15000),
    ("detention_records", 12000),
    ("family_relationships", 50000),
    ("watchlist", 8000),
    ("ecl_entries", 3000),
    ("trafficking_cases", 5000),
    ("illegal_crossings", 25000),
    ("offloading_records", 10000),
    ("risk_profiles", 150000),
    ("suspect_networks", 5000),
    ("audit_log", 100000),
]

print(f"{'Table':<30} {'Actual':>10} {'Expected':>10} {'Status':>8}")
print("─" * 62)

total_actual = 0
for table, expected in EXPECTED:
    cursor.execute(f"SELECT COUNT(*) FROM {table}")
    actual = cursor.fetchone()[0]
    total_actual += actual
    status = "✓" if actual > 0 else "✗ EMPTY"
    print(f"  {table:<28} {actual:>10,} {f'~{expected:,}':>10} {status:>8}")

print("─" * 62)
print(f"  {'TOTAL':<28} {total_actual:>10,}")
print(f"\n{'✅ Database ready!' if total_actual > 1000000 else '⚠ Check errors above.'}")

Table                              Actual   Expected   Status
──────────────────────────────────────────────────────────────
  countries                            50        ~50        ✓
  ports_of_entry                       39        ~39        ✓
  visa_categories                      20        ~20        ✓
  sponsors                             90       ~100        ✓
  travelers                       150,000   ~150,000        ✓
  document_registry               219,240   ~200,000        ✓
  visa_applications               250,000   ~250,000        ✓
  travel_records                  500,000   ~500,000        ✓
  asylum_claims                    30,000    ~30,000        ✓
  removal_orders                   15,000    ~15,000        ✓
  detention_records                12,000    ~12,000        ✓
  family_relationships             48,713    ~50,000        ✓
  watchlist                         8,000     ~8,000        ✓
  ecl_entries                       2,676     ~3,000        ✓
  traff

## Cell 34: Quick Sanity Check — Sample Queries

In [39]:
# Top 5 nationalities by traveler count
query = """
SELECT c.country_name, COUNT(*) AS cnt
FROM travelers t
JOIN countries c ON t.nationality_id = c.country_id
GROUP BY c.country_name
ORDER BY cnt DESC
FETCH FIRST 5 ROWS ONLY
"""
df_test = pd.read_sql(query, conn)
print("Top 5 nationalities:")
print(df_test.to_string(index=False))

print("\n---\n")

# Travel records by direction
query2 = """
SELECT travel_direction, COUNT(*) AS cnt
FROM travel_records
GROUP BY travel_direction
ORDER BY cnt DESC
"""
df_test2 = pd.read_sql(query2, conn)
print("Travel records by direction:")
print(df_test2.to_string(index=False))

Top 5 nationalities:
COUNTRY_NAME   CNT
    Pakistan 59817
 Afghanistan 22284
       India 14983
  Bangladesh  9016
       Syria  4533

---

Travel records by direction:
TRAVEL_DIRECTION    CNT
         Inbound 225448
        Outbound 224223
         Transit  50329


/tmp/ipykernel_10033/2830789157.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql(query, conn)
/tmp/ipykernel_10033/2830789157.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test2 = pd.read_sql(query2, conn)


## Cell 35: Close Connection

In [40]:
conn.close()
print("Connection closed.")

Connection closed.
